In [ ]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)

# NWB and Spikeinterface

## The NWB Ecocsystem

[Link to slides](https://docs.google.com/presentation/d/1DwUEKQrUkTLi2hm4RwV1egWnJ3wsGUI973Zku-HA-zg/edit?usp=sharing)

## Neurconv

## Adding Spikeinterface objects to NWB

Let's start by generating some artificial data

In [ ]:
from spikeinterface.core import generate_ground_truth_recording


recording, sorting = generate_ground_truth_recording(num_channels=8, num_units=3, durations=[60*10.0], seed=0,)

recording = recording.rename_channels(new_channel_ids=["A", "B", "C", "D", "E", "F", "G", "H"]) 
sorting = sorting.rename_units(new_unit_ids=["Unit A", "Unit B", "Unit C"])

# Add new properties to the recording and sorting
recording.set_property(key="a_channel_property", values=[f"property {channel}" for channel in recording.get_channel_ids()])
recording.set_property(key="brain_area", values=[f"Area {channel}" for channel in recording.get_channel_ids()])
sorting.set_property(key="a_unit_property", values=["property Unit A", "property Unit B", "property Unit C"])


In [ ]:
recording

In [ ]:
from probeinterface.plotting import plot_probe
probe = recording.get_probe()

plot_probe(probe);

### Recording

In [ ]:
from neuroconv.tools.spikeinterface import add_electrical_series
from pynwb.testing.mock.file import mock_NWBFile


nwbfile = mock_NWBFile()
nwbfile

In [ ]:
add_electrical_series(recording=recording, nwbfile=nwbfile)
nwbfile

In [ ]:
nwbfile.electrodes.to_dataframe()

In [ ]:
from spikeinterface.preprocessing import bandpass_filter

filtered_recording = bandpass_filter(recording, freq_min=1, freq_max=300)

add_electrical_series(recording=filtered_recording, nwbfile=nwbfile, write_as="lfp")
nwbfile

### Sorting

In [ ]:
from neuroconv.tools.spikeinterface import add_sorting


add_sorting(sorting=sorting, nwbfile=nwbfile, skip_properties=["gt_unit_locations"])
nwbfile

In [ ]:
nwbfile.units.to_dataframe()

In [ ]:
# Save this to a file
from pathlib import Path
from pynwb import NWBHDF5IO

nwb_file_path = Path.cwd() / "nwb_to_spikeinterface"
with NWBHDF5IO(path=nwb_file_path, mode="w") as io:
    io.write(nwbfile)

### Sorting analyzer (Probably not)

## Loading NWB files in Spikeinterface

In [ ]:
from spikeinterface.extractors import NwbSortingExtractor, NwbRecordingExtractor

recording = NwbRecordingExtractor(file_path=nwb_file_path)

In [ ]:
recording = NwbRecordingExtractor(file_path=nwb_file_path, electrical_series_path="acquisition/ElectricalSeriesRaw")
recording

In [ ]:
from probeinterface.plotting import plot_probe
probe = recording.get_probe()

plot_probe(probe);

### Streaming objects from NWB

In [ ]:
from dandi.dandiapi import DandiAPIClient
from spikeinterface.extractors import NwbRecordingExtractor, IblSortingExtractor
from one.api import ONE

client = DandiAPIClient.for_dandi_instance("dandi")

dandiset_id = "000409"
dandiset = client.get_dandiset(dandiset_id)


#asset_path = dandiset_paths_with_ecephys[3]
asset_path = "sub-KS042/sub-KS042_ses-8c552ddc-813e-4035-81cc-3971b57efe65_behavior+ecephys+image.nwb"
recording_asset = dandiset.get_asset_by_path(path=asset_path)
url = recording_asset.get_content_url(follow_redirects=True, strip_query=True)
file_path = url



electrical_series_path = "acquisition/ElectricalSeriesAp00"
recording = NwbRecordingExtractor(file_path=file_path, stream_mode="remfile", electrical_series_path=electrical_series_path)  

In [ ]:
%matplotlib widget

from spikeinterface.widgets import plot_traces


plot_traces(recording=recording, backend="ipywidgets")

In [ ]:
session_id = recording._file["general"]["session_id"][()].decode()
eid = session_id.split("-chunking")[0] # eid : experiment id



ONE.setup(base_url='https://openalyx.internationalbrainlab.org', silent=True)
one_instance = ONE(password='international')


pids, probes = one_instance.eid2pid(eid)


# Let's select the probe 

probe_number = electrical_series_path.split("Ap")[-1]

sorting_pid = None 
for pid, probe in zip(pids, probes):
    probe_number_in_pid = probe[-2:]
    if probe_number_in_pid == probe_number:
        sorting_pid = pid
        break
    


sorting = IblSortingExtractor(pid=sorting_pid, one=one_instance, good_clusters_only=True)
sorting

In [ ]:
from spikeinterface.widgets import plot_rasters


plot_rasters(sorting=sorting, backend="matplotlib")
